# Fully-Connected Neural Network

## Data Preparation

In [ ]:
# Import Necessary Packages and Mount Drive
!pip install scikit-learn --upgrade
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import KFold, RandomizedSearchCV
pd.set_option('display.max_columns', None)

from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
%cd '/content/gdrive/MyDrive/ECSE_552/Project'

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split
from torchvision import transforms

!pip install pytorch_lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.utilities import rank_zero_only
from pytorch_lightning.loggers import LightningLoggerBase
from pytorch_lightning.loggers.base import rank_zero_experiment
from collections import defaultdict

In [ ]:
# Create checkpoint callback that monitors validation loss
class LitAutoEncoder(pl.LightningModule):
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.backbone(x)

        # 1. calculate loss
        loss = F.cross_entropy(y_hat, y)

        # 2. log `val_loss`
        self.log('val_loss', loss)


# Logger
class DictLogger(LightningLoggerBase):

    def __init__(self):
        super().__init__()
        def def_value(): 
            return []
              
        # Defining the dict 
        self.metrics = defaultdict(def_value) 


    @property
    def name(self):
        return 'DictLogger'

    @property
    @rank_zero_experiment
    def experiment(self):
        # Return the experiment object associated with this logger.
        pass

    @property
    def version(self):
        # Return the experiment version, int or str.
        return '0.1'

    @rank_zero_only
    def log_hyperparams(self, params):
        # params is an argparse.Namespace
        # your code to record hyperparameters goes here
        pass

    @rank_zero_only
    def log_metrics(self, metrics, step):
        # metrics is a dictionary of metric names and values
        # your code to record metrics goes here
        for key in metrics.keys():
            self.metrics[key].append(metrics[key])

    @rank_zero_only
    def save(self):
        # Optional. Any code necessary to save logger data goes here
        # If you implement this, remember to call `super().save()`
        # at the start of the method (important for aggregation of metrics)
        super().save()

    @rank_zero_only
    def finalize(self, status):
        # Optional. Any code that needs to be run after training
        # finishes goes here
        pass

In [ ]:
# RMSELoss
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-10):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self,yhat,y):
        loss = torch.sqrt(self.mse(yhat,y) + self.eps)
        return loss

In [ ]:
# Import the dataset
df = pd.read_csv('data/ON_demand_weather_17-20.csv', index_col=0)


# Do all the preprocessing required
from preprocessing.preprocessing import one_hot_encode_days, one_hot_encode_months, time_differencing, split_and_scale, sin_cos_waves, sliding_windows


# Remove weather stations and market demand columns
df.drop(['Market Demand',
         'toronto_Temp (C)',
         'hamilton_Temp (C)',
         'ottawa_Temp (C)',
         'kitchener_Temp (C)',
         'london_Temp (C)',
         'windsor_Temp (C)'], axis=1, inplace=True)

# Apply scaling if desired and split dataset into train, validation, and testset
scaler = 'standard'  # Define the type of scaler, options are 'min-max', 'standard', and None
columns_to_scale = ['Ontario Demand', 'Weighted Average Temp (C)']  # Define columns to be scaled, add time-differenced column if applicable
target_column = columns_to_scale[0]  # Define target column (required to allow unscaling later), change to time-differenced column if applicable
vali_set_start_date='2019-07-01'  # First day of validation set
test_set_start_date='2020-01-01'  # First day of test set

train_df, vali_df, test_df, target_scaler = split_and_scale(df,
                                                            scaler=scaler,
                                                            columns_to_scale=columns_to_scale,
                                                            target_column=target_column,
                                                            vali_set_start_date=vali_set_start_date,
                                                            test_set_start_date=test_set_start_date)


## Model Training and Evaluation

#### 1Hour Prediction

In [ ]:
# Create sliding window for 1H predictions
window_size = 24
flatten = True
output_window_size = 1
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

# Adjust target vectors, keep only last element
y_train = y_train[:, -1].reshape(-1,1)
y_vali = y_vali[:, -1].reshape(-1,1)

BATCH_SIZE = 32
# Convert to tensor
x_train_tensor = torch.FloatTensor(x_train)
y_train_tensor = torch.FloatTensor(y_train)
x_vali_tensor = torch.FloatTensor(x_vali)
y_vali_tensor = torch.FloatTensor(y_vali)
x_test_tensor = torch.FloatTensor(x_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)
valid_data = TensorDataset(x_vali_tensor, y_vali_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

# Create DataLoader
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)

In [ ]:
# Hyperparams
n_inputs = x_train.shape[1]
output_size = 1
n_neuron = 100
max_epochs = 200

# Lightning Module
class fcnn(pl.LightningModule):
  def __init__(self, n_inputs, hidden_layer_size, output_size):
    super().__init__()
    self.n_inputs = n_inputs
    self.output_size = output_size

    self.input_layer = torch.nn.Linear(n_inputs, hidden_layer_size)
    self.layer_h1 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h2 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h3 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    self.output_layer = torch.nn.Linear(hidden_layer_size, output_size)

  def forward(self, x):
    batch_size, n_features = x.size()
    x = x.view(batch_size, -1)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.layer_h1(x))
    # x = F.relu(self.layer_h2(x))
    # x = F.relu(self.layer_h3(x))
    output = self.output_layer(x)
    return output
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, min_lr=1e-7)
    return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    x_hat = self(x)
    train_loss = F.mse_loss(x_hat, y)
    self.log('train_loss', train_loss, on_step=False, on_epoch=True)
    return train_loss
  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    x_hat = self(x)
    val_loss = F.mse_loss(x_hat, y)
    self.log('val_loss', val_loss)


fcnn_model = fcnn(n_inputs, n_neuron, output_size)

# training
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor='val_loss')

# Learning rate monitor
lr_monitor = LearningRateMonitor(logging_interval='epoch')
fcnn_logger = DictLogger()
early_stopping = EarlyStopping('val_loss', verbose=True, min_delta=0.0001, patience=5)
trainer = pl.Trainer(max_epochs=max_epochs, gpus=1, progress_bar_refresh_rate=30, logger=fcnn_logger, callbacks=[early_stopping,checkpoint_callback, lr_monitor],
                    default_root_dir='/content/gdrive/MyDrive/ECSE_552/Project/Checkpoints/fcnn')
start_time = time.time()
trainer.fit(fcnn_model, train_dataloader, valid_dataloader)
fcnn_training_time = time.time() - start_time

##### Test

In [ ]:
dataset = valid_dataloader
if target_scaler is not None:
  for x,y in dataset:
    x_data = x
    y_data = y

  train_pred = fcnn_model(x_train_tensor).detach()
  pred = fcnn_model(x_data).detach()

  train_unscaled_pred = target_scaler.inverse_transform(train_pred)
  train_unscaled_y_data = target_scaler.inverse_transform(y_train_tensor)

  unscaled_pred = target_scaler.inverse_transform(pred)
  unscaled_y_data = target_scaler.inverse_transform(y_data)

  train_loss_unscaled = sqrt(mean_squared_error(train_unscaled_y_data, train_unscaled_pred))
  vali_loss_unscaled = sqrt(mean_squared_error(unscaled_y_data, unscaled_pred))

  print('Unscaled Train RMSE: {:.3f}'.format(train_loss_unscaled))
  print('Unscaled Vali RMSE: {:.3f}'.format(vali_loss_unscaled))

dataset = test_dataloader
if target_scaler is not None:
  for x,y in dataset:
    x_data = x
    y_data = y


  pred = fcnn_model(x_data).detach()

  unscaled_pred = target_scaler.inverse_transform(pred)
  unscaled_y_data = target_scaler.inverse_transform(y_data)

  test_loss_unscaled = sqrt(mean_squared_error(unscaled_y_data, unscaled_pred))
  test_mape = mean_absolute_percentage_error(unscaled_y_data, unscaled_pred)


  print('Unscaled Test RMSE: {:.3f}'.format(test_loss_unscaled))
  print('Test MAPE: {:.4f}'.format(test_mape))

In [ ]:
# Plot RF performance
plt.figure(figsize=(10,2))
plt.plot(unscaled_pred[1:25], label='pred')
plt.plot(unscaled_y_data[1:25], label='true')
plt.legend()
plt.xlabel('hours')
plt.ylabel('Demand in MW')
plt.title('FCNN prediction example on validation set')
plt.show()

#### 24Hour Sequence Prediction 

In [ ]:
# Create sliding window for 1H predictions
window_size = 24
flatten = True
output_window_size = 24
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

# Adjust target vectors, keep only last element
y_train = y_train[:, -1].reshape(-1,1)
y_vali = y_vali[:, -1].reshape(-1,1)

BATCH_SIZE = 32
# Convert to tensor
x_train_tensor = torch.FloatTensor(x_train)
y_train_tensor = torch.FloatTensor(y_train)
x_vali_tensor = torch.FloatTensor(x_vali)
y_vali_tensor = torch.FloatTensor(y_vali)
x_test_tensor = torch.FloatTensor(x_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)
valid_data = TensorDataset(x_vali_tensor, y_vali_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

# Create DataLoader
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)

In [ ]:
# Hyperparams
n_inputs = x_train.shape[1]
output_size = 1
n_neuron = 100
max_epochs = 200

# Lightning Module
class fcnn(pl.LightningModule):
  def __init__(self, n_inputs, hidden_layer_size, output_size):
    super().__init__()
    self.n_inputs = n_inputs
    self.output_size = output_size

    self.input_layer = torch.nn.Linear(n_inputs, hidden_layer_size)
    self.layer_h1 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h2 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h3 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    self.output_layer = torch.nn.Linear(hidden_layer_size, output_size)

  def forward(self, x):
    batch_size, n_features = x.size()
    x = x.view(batch_size, -1)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.layer_h1(x))
    # x = F.relu(self.layer_h2(x))
    # x = F.relu(self.layer_h3(x))
    output = self.output_layer(x)
    return output
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, min_lr=1e-7)
    return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    x_hat = self(x)
    train_loss = F.mse_loss(x_hat, y)
    self.log('train_loss', train_loss, on_step=False, on_epoch=True)
    return train_loss
  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    x_hat = self(x)
    val_loss = F.mse_loss(x_hat, y)
    self.log('val_loss', val_loss)


fcnn_model = fcnn(n_inputs, n_neuron, output_size)

# training
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor='val_loss')

# Learning rate monitor
lr_monitor = LearningRateMonitor(logging_interval='epoch')
fcnn_logger = DictLogger()
early_stopping = EarlyStopping('val_loss', verbose=True, min_delta=0.0001, patience=5)
trainer = pl.Trainer(max_epochs=max_epochs, gpus=1, progress_bar_refresh_rate=30, logger=fcnn_logger, callbacks=[early_stopping,checkpoint_callback, lr_monitor],
                    default_root_dir='/content/gdrive/MyDrive/ECSE_552/Project/Checkpoints/fcnn')
start_time = time.time()
trainer.fit(fcnn_model, train_dataloader, valid_dataloader)
fcnn_training_time = time.time() - start_time

##### Test

In [ ]:
dataset = valid_dataloader
if target_scaler is not None:
  for x,y in dataset:
    x_data = x
    y_data = y

  train_pred = fcnn_model(x_train_tensor).detach()
  pred = fcnn_model(x_data).detach()

  train_unscaled_pred = target_scaler.inverse_transform(train_pred)
  train_unscaled_y_data = target_scaler.inverse_transform(y_train_tensor)

  unscaled_pred = target_scaler.inverse_transform(pred)
  unscaled_y_data = target_scaler.inverse_transform(y_data)

  train_loss_unscaled = sqrt(mean_squared_error(train_unscaled_y_data, train_unscaled_pred))
  vali_loss_unscaled = sqrt(mean_squared_error(unscaled_y_data, unscaled_pred))

  print('Unscaled Train RMSE: {:.3f}'.format(train_loss_unscaled))
  print('Unscaled Vali RMSE: {:.3f}'.format(vali_loss_unscaled))

dataset = test_dataloader
if target_scaler is not None:
  for x,y in dataset:
    x_data = x
    y_data = y


  pred = fcnn_model(x_data).detach()

  unscaled_pred = target_scaler.inverse_transform(pred)
  unscaled_y_data = target_scaler.inverse_transform(y_data)

  test_loss_unscaled = sqrt(mean_squared_error(unscaled_y_data, unscaled_pred))
  test_mape = mean_absolute_percentage_error(unscaled_y_data, unscaled_pred)


  print('Unscaled Test RMSE: {:.3f}'.format(test_loss_unscaled))
  print('Test MAPE: {:.4f}'.format(test_mape))
  
# Get last value of label sequence
last_true_value = y_vali[:,-1]

# Get last value of predicted sequence
vali_pred = fcnn_model(x_vali_tensor).detach()
last_pred_value = vali_pred[:,-1]

# Unscale
if scaler is not None:
  last_pred_value_unscaled = target_scaler.inverse_transform(last_pred_value)
  last_true_value_unscaled = target_scaler.inverse_transform(last_true_value)

# Compute loss on last prediction
last_loss = mean_squared_error(last_true_value, last_pred_value)
print('RMSE on last time step of predicted sequence: {:.3f}'.format(sqrt(last_loss)))
if scaler is not None:
  last_loss_unscaled = mean_squared_error(last_true_value_unscaled, last_pred_value_unscaled)
  print('RMSE (unscaled) on last time step of predicted sequence: {:.3f}'.format(sqrt(last_loss_unscaled)))

#### 24Hour AutoRegressive
The 24H AutoRegressive Model for FCNN is based on the 1H model.

In [ ]:
# Create sliding window for 1H predictions
window_size = 24
flatten = True
output_window_size = 1
perform_feature_shift = False

# Sliding windows
# note that the function removes the 'Date' column (not numeric)
x_train, y_train = sliding_windows(df=train_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_vali, y_vali = sliding_windows(df=vali_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)
x_test, y_test = sliding_windows(df=test_df, window_size=window_size, target_column=target_column, flatten=flatten, output_window_size=output_window_size, perform_feature_shift=perform_feature_shift)

# Adjust target vectors, keep only last element
y_train = y_train[:, -1].reshape(-1,1)
y_vali = y_vali[:, -1].reshape(-1,1)

BATCH_SIZE = 32
# Convert to tensor
x_train_tensor = torch.FloatTensor(x_train)
y_train_tensor = torch.FloatTensor(y_train)
x_vali_tensor = torch.FloatTensor(x_vali)
y_vali_tensor = torch.FloatTensor(y_vali)
x_test_tensor = torch.FloatTensor(x_test)
y_test_tensor = torch.FloatTensor(y_test)

# Create Dataset
train_data = TensorDataset(x_train_tensor, y_train_tensor)
valid_data = TensorDataset(x_vali_tensor, y_vali_tensor)
test_data = TensorDataset(x_test_tensor, y_test_tensor)

# Create DataLoader
train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_data, batch_size=len(valid_data), shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)

In [ ]:
# Hyperparams
n_inputs = x_train.shape[1]
output_size = 1
n_neuron = 100
max_epochs = 200

# Lightning Module
class fcnn(pl.LightningModule):
  def __init__(self, n_inputs, hidden_layer_size, output_size):
    super().__init__()
    self.n_inputs = n_inputs
    self.output_size = output_size

    self.input_layer = torch.nn.Linear(n_inputs, hidden_layer_size)
    self.layer_h1 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h2 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    # self.layer_h3 = torch.nn.Linear(hidden_layer_size, hidden_layer_size)
    self.output_layer = torch.nn.Linear(hidden_layer_size, output_size)

  def forward(self, x):
    batch_size, n_features = x.size()
    x = x.view(batch_size, -1)
    x = F.relu(self.input_layer(x))
    x = F.relu(self.layer_h1(x))
    # x = F.relu(self.layer_h2(x))
    # x = F.relu(self.layer_h3(x))
    output = self.output_layer(x)
    return output
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=2, min_lr=1e-7)
    return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    x_hat = self(x)
    train_loss = F.mse_loss(x_hat, y)
    self.log('train_loss', train_loss, on_step=False, on_epoch=True)
    return train_loss
  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    x_hat = self(x)
    val_loss = F.mse_loss(x_hat, y)
    self.log('val_loss', val_loss)


fcnn_model = fcnn(n_inputs, n_neuron, output_size)

# training
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor='val_loss')

# Learning rate monitor
lr_monitor = LearningRateMonitor(logging_interval='epoch')
fcnn_logger = DictLogger()
early_stopping = EarlyStopping('val_loss', verbose=True, min_delta=0.0001, patience=5)
trainer = pl.Trainer(max_epochs=max_epochs, gpus=1, progress_bar_refresh_rate=30, logger=fcnn_logger, callbacks=[early_stopping,checkpoint_callback, lr_monitor],
                    default_root_dir='/content/gdrive/MyDrive/ECSE_552/Project/Checkpoints/fcnn')
start_time = time.time()
trainer.fit(fcnn_model, train_dataloader, valid_dataloader)
fcnn_training_time = time.time() - start_time

In [ ]:
# AutoRegressive Part
pred_length = 24  # number of steps done per prediction
dataset = test_df.copy()  # define dataset (before sliding window)
dataset.drop(columns='Date', inplace=True)
dataset[target_column] = dataset.pop(target_column)
dataset = torch.FloatTensor(dataset.to_numpy())

# Create dataframe to store losses
loss_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])
loss_unscaled_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])
loss_mape_df = pd.DataFrame(columns=[j for j in range(1, pred_length+1)])

true_values = []

for i in range(dataset.shape[0] - window_size - pred_length+1):
  # Get first query for each sequence
  query = dataset[i:i+window_size].reshape(dataset.shape[1]*window_size)
  # print(query)
  for j in range(pred_length):
    # Predict
    pred = fcnn_model(query.reshape(1,-1)).detach()

    # Extract true value + additional data from next row
    next_row = dataset[i+j+window_size:i+j+window_size+1]
    true_value = next_row[:,-1]
    additional_data = next_row[:,:-1][0]

    # Compute loss and add to dataframes
    loss = mean_squared_error(pred.detach(), true_value.detach())
    loss_unscaled = mean_squared_error(target_scaler.inverse_transform(pred.detach()), target_scaler.inverse_transform(true_value.detach()))
    loss_mape = mean_absolute_percentage_error(target_scaler.inverse_transform(true_value.detach()), target_scaler.inverse_transform(pred.detach()))
    loss_df.loc[i, j+1] = loss
    loss_unscaled_df.loc[i, j+1] = loss_unscaled
    loss_mape_df.loc[i, j + 1] = loss_mape
    additional_data = torch.squeeze(additional_data,-1)
    pred = torch.squeeze(pred, -1)
    # Create new query window from data + prediction
    new_row = torch.cat((additional_data, pred),0)
    query = torch.cat((query[dataset.shape[1]:], new_row), 0)

##### Test

In [ ]:
print('Sequence RMSE: {:.3f}'.format(sqrt(loss_unscaled_df.mean().mean())))
print('Sequence MAPE: {:.4f}'.format(loss_mape_df.mean().mean()))

In [ ]:
# GET SQUAREROOT OF VALUES
# CURRENTLY REPORTING MSE BELOW
# Plot results
plt.title('FCNN autoregressive error')
plt.ylabel('RMSE')
plt.xlabel('Forecasted time-steps')
plt.plot(loss_unscaled_df.mean())
plt.show()